# Corporación Favorita Grocery Sales Forecasting
## d02_w01_EDA_data_loading_filtering.ipynb

**Author:** Alberto Diaz Durana  
**Date:** November 2025  
**Purpose:** Filter train.csv to Guayas region and top-3 families, create 300K sample for EDA

---

## Objectives

This notebook accomplishes the following:

- Load train.csv with Dask and filter to Guayas stores (11 stores)
- Merge with items.csv to get product family information
- Filter to top-3 families: GROCERY I, BEVERAGES, CLEANING
- Random sample 300,000 rows for development speed
- Export filtered dataset as guayas_sample_300k.csv and .pkl
- Validate filtering results and document reduction

---

## Business Context

**Why this filtering matters:**

Focusing on Guayas region and top-3 families enables:
- Manageable dataset size (300K vs 125M rows)
- Representative patterns (58.4% of items, 20.4% of stores)
- Faster iteration during EDA and modeling
- Regional insights for Guayas market specifically

**Filtering criteria:**
- Guayas stores: 11 of 54 (store IDs: 24, 26, 27, 28, 29, 30, 32, 34, 35, 36, 51)
- Top-3 families: GROCERY I (1,334 items), BEVERAGES (613 items), CLEANING (446 items)
- Sample: 300,000 random rows (reproducible with seed=42)

**Deliverables:**
- guayas_sample_300k.csv (300K rows, filtered dataset)
- guayas_sample_300k.pkl (faster loading for Day 3)
- Filtering validation report

---

## Input Dependencies

From Day 1:
- train.csv (125,497,040 rows)
- stores.csv (54 stores)
- items.csv (4,100 items)
- Guayas store list: [24, 26, 27, 28, 29, 30, 32, 34, 35, 36, 51]
- Top-3 families: ['GROCERY I', 'BEVERAGES', 'CLEANING']

---

## 1. Setup & Imports

**Objective:** Import libraries, load support files from Day 1, configure paths

**Activities:**
- Import pandas, dask, numpy
- Load inventory results from Day 1 (stores, items)
- Define path constants
- Set random seed for reproducibility

**Expected output:** Environment ready, support files reloaded

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import dask
import dask.dataframe as dd
from pathlib import Path
import warnings

# Package versions
print("Package Versions:")
print(f"  pandas: {pd.__version__}")
print(f"  numpy: {np.__version__}")
print(f"  dask: {dask.__version__}")

# Configure environment
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

print("\nOK - Environment configured")

Package Versions:
  pandas: 2.1.4
  numpy: 1.26.4
  dask: 2025.11.0

OK - Environment configured


In [2]:
# Determine paths (works from notebooks/ or project root)
current_dir = Path(__file__).parent if '__file__' in globals() else Path.cwd()
project_root = current_dir.parent if current_dir.name == 'notebooks' else current_dir

# Define path constants
DATA_RAW = project_root / 'data' / 'raw'
DATA_PROCESSED = project_root / 'data' / 'processed'

# Verify paths exist
assert DATA_RAW.exists(), f"ERROR - Path not found: {DATA_RAW}"
assert DATA_PROCESSED.exists(), f"ERROR - Path not found: {DATA_PROCESSED}"

print("OK - Paths validated:")
print(f"  Project root: {project_root.resolve()}")
print(f"  DATA_RAW: {DATA_RAW.resolve()}")
print(f"  DATA_PROCESSED: {DATA_PROCESSED.resolve()}")

# Set random seed for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
print(f"\nRandom seed set: {RANDOM_SEED}")

OK - Paths validated:
  Project root: C:\Users\adiaz\OneDrive\Dokumente\PythonScripts\MasterClass\Demand-forecasting-in-retail
  DATA_RAW: C:\Users\adiaz\OneDrive\Dokumente\PythonScripts\MasterClass\Demand-forecasting-in-retail\data\raw
  DATA_PROCESSED: C:\Users\adiaz\OneDrive\Dokumente\PythonScripts\MasterClass\Demand-forecasting-in-retail\data\processed

Random seed set: 42


In [3]:
# Load support files from Day 1
print("Loading support files...")

df_stores = pd.read_csv(DATA_RAW / 'stores.csv')
df_items = pd.read_csv(DATA_RAW / 'items.csv')

print(f"✓ stores.csv: {len(df_stores)} stores")
print(f"✓ items.csv: {len(df_items)} items")

# Define Guayas scope (from Day 1 analysis)
guayas_store_nbrs = [24, 26, 27, 28, 29, 30, 32, 34, 35, 36, 51]
top_3_families = ['GROCERY I', 'BEVERAGES', 'CLEANING']

print(f"\nGuayas scope defined:")
print(f"  Stores: {len(guayas_store_nbrs)} stores")
print(f"  Store IDs: {guayas_store_nbrs}")
print(f"  Families: {top_3_families}")

# Verify items in top-3 families
top_3_items = df_items[df_items['family'].isin(top_3_families)]
print(f"  Items in top-3 families: {len(top_3_items):,}")

print("\nOK - Support files loaded and scope defined")

Loading support files...
✓ stores.csv: 54 stores
✓ items.csv: 4100 items

Guayas scope defined:
  Stores: 11 stores
  Store IDs: [24, 26, 27, 28, 29, 30, 32, 34, 35, 36, 51]
  Families: ['GROCERY I', 'BEVERAGES', 'CLEANING']
  Items in top-3 families: 2,393

OK - Support files loaded and scope defined


## 2. Load & Filter train.csv to Guayas

**Objective:** Load large train.csv with Dask and filter to Guayas stores only

**Activities:**
- Load train.csv with Dask (125M rows)
- Filter to 11 Guayas stores
- Convert filtered result to pandas
- Validate row count reduction
- Check memory usage

**Expected output:** 
- Filtered dataset with ~25M rows (20% of original)
- Pandas DataFrame ready for further filtering

In [4]:
# Load train.csv with Dask
print("Loading train.csv with Dask (125M rows)...")
print("This may take 1-2 minutes...\n")

df_train = dd.read_csv(DATA_RAW / 'train.csv')

print(f"OK - train.csv loaded (Dask DataFrame)")
print(f"  Columns: {list(df_train.columns)}")
print(f"  Estimated rows: 125,497,040")

Loading train.csv with Dask (125M rows)...
This may take 1-2 minutes...

OK - train.csv loaded (Dask DataFrame)
  Columns: ['id', 'date', 'store_nbr', 'item_nbr', 'unit_sales', 'onpromotion']
  Estimated rows: 125,497,040


In [5]:
# Filter to Guayas stores only
print("Filtering to Guayas stores (11 stores)...")
print("This will take 2-3 minutes - processing 125M rows...\n")

df_train_guayas = df_train[df_train['store_nbr'].isin(guayas_store_nbrs)]

print("Computing filtered dataset...")
df_train_guayas = df_train_guayas.compute()

print(f"\nOK - Filtering complete!")
print(f"  Original rows: 125,497,040")
print(f"  Filtered rows: {len(df_train_guayas):,}")
print(f"  Reduction: {(1 - len(df_train_guayas)/125497040)*100:.1f}%")
print(f"  Memory usage: {df_train_guayas.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

Filtering to Guayas stores (11 stores)...
This will take 2-3 minutes - processing 125M rows...

Computing filtered dataset...

OK - Filtering complete!
  Original rows: 125,497,040
  Filtered rows: 22,941,656
  Reduction: 81.7%
  Memory usage: 1356.5 MB


In [6]:
# Display first rows and basic info
print("First 5 rows of Guayas-filtered data:")
print(df_train_guayas.head())

print("\nData types:")
print(df_train_guayas.dtypes)

print("\nUnique stores in filtered data:")
print(sorted(df_train_guayas['store_nbr'].unique()))

print("\nUnique items in filtered data:")
print(f"  Total unique items: {df_train_guayas['item_nbr'].nunique():,}")

First 5 rows of Guayas-filtered data:
          id        date  store_nbr  item_nbr  unit_sales  onpromotion
18789  18789  2013-01-02         24    103665        6.00          NaN
18790  18790  2013-01-02         24    105574       12.00          NaN
18791  18791  2013-01-02         24    105575        2.00          NaN
18792  18792  2013-01-02         24    105577        1.00          NaN
18793  18793  2013-01-02         24    105693        7.00          NaN

Data types:
id                       int64
date           string[pyarrow]
store_nbr                int64
item_nbr                 int64
unit_sales             float64
onpromotion            float64
dtype: object

Unique stores in filtered data:
[24, 26, 27, 28, 29, 30, 32, 34, 35, 36, 51]

Unique items in filtered data:
  Total unique items: 4,008


## 3. Filter to Top-3 Families & Sample 300K

**Objective:** Merge with items, filter to top-3 families, sample 300K rows

**Activities:**
- Merge train data with items.csv to get family column
- Filter to top-3 families: GROCERY I, BEVERAGES, CLEANING
- Random sample 300,000 rows (seed=42 for reproducibility)
- Validate final dataset characteristics

**Expected output:** 
- Final dataset: 300,000 rows
- Guayas stores + top-3 families only
- Representative sample of filtered data

In [7]:
# Merge with items to get family information
print("Merging with items.csv to get product family...")

df_train_merged = df_train_guayas.merge(
    df_items[['item_nbr', 'family', 'class', 'perishable']], 
    on='item_nbr', 
    how='left'
)

print(f"OK - Merge complete")
print(f"  Rows after merge: {len(df_train_merged):,}")
print(f"  Columns: {list(df_train_merged.columns)}")

# Check for merge issues
null_families = df_train_merged['family'].isnull().sum()
print(f"  Rows with missing family: {null_families}")

Merging with items.csv to get product family...
OK - Merge complete
  Rows after merge: 22,941,656
  Columns: ['id', 'date', 'store_nbr', 'item_nbr', 'unit_sales', 'onpromotion', 'family', 'class', 'perishable']
  Rows with missing family: 0


In [8]:
# Filter to top-3 families
print("Filtering to top-3 families...")
print(f"  Families to keep: {top_3_families}\n")

df_train_top3 = df_train_merged[df_train_merged['family'].isin(top_3_families)].copy()

print(f"OK - Filtering complete")
print(f"  Rows before family filter: {len(df_train_merged):,}")
print(f"  Rows after family filter: {len(df_train_top3):,}")
print(f"  Reduction: {(1 - len(df_train_top3)/len(df_train_merged))*100:.1f}%")

print("\nFamily distribution in filtered data:")
family_counts = df_train_top3['family'].value_counts()
print(family_counts)

print("\nPercentage by family:")
for family, count in family_counts.items():
    pct = count / len(df_train_top3) * 100
    print(f"  {family:<15} {count:>10,} ({pct:>5.1f}%)")

Filtering to top-3 families...
  Families to keep: ['GROCERY I', 'BEVERAGES', 'CLEANING']

OK - Filtering complete
  Rows before family filter: 22,941,656
  Rows after family filter: 14,745,768
  Reduction: 35.7%

Family distribution in filtered data:
family
GROCERY I    8398042
BEVERAGES    3249089
CLEANING     3098637
Name: count, dtype: int64

Percentage by family:
  GROCERY I        8,398,042 ( 57.0%)
  BEVERAGES        3,249,089 ( 22.0%)
  CLEANING         3,098,637 ( 21.0%)


In [9]:
# Random sample 300,000 rows
print("Sampling 300,000 random rows...")
print(f"  Random seed: {RANDOM_SEED}")
print(f"  Sample fraction: {300000/len(df_train_top3)*100:.3f}%\n")

df_sample = df_train_top3.sample(n=300000, random_state=RANDOM_SEED).copy()

# Reset index for clean sequential numbering
df_sample = df_sample.reset_index(drop=True)

print(f"OK - Sampling complete")
print(f"  Sample size: {len(df_sample):,} rows")
print(f"  Columns: {len(df_sample.columns)}")
print(f"  Memory usage: {df_sample.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

print("\nFamily distribution in sample:")
sample_family_counts = df_sample['family'].value_counts()
for family, count in sample_family_counts.items():
    pct = count / len(df_sample) * 100
    print(f"  {family:<15} {count:>7,} ({pct:>5.1f}%)")

Sampling 300,000 random rows...
  Random seed: 42
  Sample fraction: 2.034%

OK - Sampling complete
  Sample size: 300,000 rows
  Columns: 9
  Memory usage: 36.6 MB

Family distribution in sample:
  GROCERY I       170,594 ( 56.9%)
  BEVERAGES        66,147 ( 22.0%)
  CLEANING         63,259 ( 21.1%)


In [10]:
# Display sample characteristics
print("Sample dataset overview:")
print("\nFirst 5 rows:")
print(df_sample.head())

print("\nLast 5 rows:")
print(df_sample.tail())

print("\nBasic statistics for unit_sales:")
print(df_sample['unit_sales'].describe())

print("\nStores in sample:")
store_counts = df_sample['store_nbr'].value_counts().sort_index()
print(f"  Unique stores: {df_sample['store_nbr'].nunique()}")
print(f"  All 11 Guayas stores present: {set(df_sample['store_nbr'].unique()) == set(guayas_store_nbrs)}")

print("\nDate range in sample:")
print(f"  First date: {df_sample['date'].min()}")
print(f"  Last date: {df_sample['date'].max()}")

Sample dataset overview:

First 5 rows:
         id        date  store_nbr  item_nbr  unit_sales  onpromotion  \
0  86626618  2016-07-31         32   1395693        1.00         0.00   
1  53995064  2015-08-15         24   1464210       30.00         0.00   
2  21834951  2014-04-04         27   1102970        1.00         0.00   
3  14853390  2013-12-02         26   1162382       20.00          NaN   
4  14652646  2013-11-28         34    577741        2.00          NaN   

      family  class  perishable  
0  GROCERY I   1028           0  
1  BEVERAGES   1114           0  
2  BEVERAGES   1122           0  
3  GROCERY I   1022           0  
4  GROCERY I   1032           0  

Last 5 rows:
               id        date  store_nbr  item_nbr  unit_sales  onpromotion  \
299995   29636439  2014-08-25         27    219147        4.00         0.00   
299996  117116855  2017-05-28         34   1463770        2.00         0.00   
299997   30368924  2014-09-06         26    269287       15.00    

## 4. Export & Validate

**Objective:** Save filtered sample and create validation summary

**Activities:**
- Export to CSV (guayas_sample_300k.csv)
- Export to pickle (guayas_sample_300k.pkl) for faster loading
- Create filtering summary report
- Document data quality observations

**Expected output:** 
- Two export files in data/processed/
- Validation report confirming scope and quality

In [11]:
# Export to CSV
csv_path = DATA_PROCESSED / 'guayas_sample_300k.csv'
print(f"Exporting to CSV: {csv_path.name}")
df_sample.to_csv(csv_path, index=False)
print(f"  ✓ CSV saved ({csv_path.stat().st_size / 1024**2:.1f} MB)")

# Export to pickle (faster loading)
pkl_path = DATA_PROCESSED / 'guayas_sample_300k.pkl'
print(f"\nExporting to pickle: {pkl_path.name}")
df_sample.to_pickle(pkl_path)
print(f"  ✓ Pickle saved ({pkl_path.stat().st_size / 1024**2:.1f} MB)")

print("\nOK - Export complete")
print(f"  CSV: {csv_path.resolve()}")
print(f"  Pickle: {pkl_path.resolve()}")

Exporting to CSV: guayas_sample_300k.csv
  ✓ CSV saved (16.0 MB)

Exporting to pickle: guayas_sample_300k.pkl
  ✓ Pickle saved (20.6 MB)

OK - Export complete
  CSV: C:\Users\adiaz\OneDrive\Dokumente\PythonScripts\MasterClass\Demand-forecasting-in-retail\data\processed\guayas_sample_300k.csv
  Pickle: C:\Users\adiaz\OneDrive\Dokumente\PythonScripts\MasterClass\Demand-forecasting-in-retail\data\processed\guayas_sample_300k.pkl


In [12]:
# Create filtering summary report
filtering_summary = {
    'original_rows': 125_497_040,
    'after_guayas_filter': 22_941_656,
    'after_family_filter': 14_745_768,
    'final_sample': 300_000,
    'guayas_stores': len(guayas_store_nbrs),
    'top_3_families': len(top_3_families),
    'unique_items_in_sample': df_sample['item_nbr'].nunique(),
    'date_range_start': str(df_sample['date'].min()),
    'date_range_end': str(df_sample['date'].max()),
    'missing_onpromotion': df_sample['onpromotion'].isnull().sum(),
    'missing_onpromotion_pct': f"{df_sample['onpromotion'].isnull().sum() / len(df_sample) * 100:.2f}%",
    'negative_sales_count': (df_sample['unit_sales'] < 0).sum(),
    'negative_sales_pct': f"{(df_sample['unit_sales'] < 0).sum() / len(df_sample) * 100:.2f}%",
}

print("FILTERING SUMMARY")
print("=" * 70)
for key, value in filtering_summary.items():
    print(f"{key:<30} {value}")

print("\n" + "=" * 70)
print("DATA REDUCTION PIPELINE:")
print(f"  125,497,040 rows (original)")
print(f"  → 22,941,656 rows (Guayas filter, -81.7%)")
print(f"  → 14,745,768 rows (top-3 families, -35.7%)")
print(f"  → 300,000 rows (random sample, -98.0%)")
print(f"  Final: 0.24% of original dataset")

FILTERING SUMMARY
original_rows                  125497040
after_guayas_filter            22941656
after_family_filter            14745768
final_sample                   300000
guayas_stores                  11
top_3_families                 3
unique_items_in_sample         2296
date_range_start               2013-01-02
date_range_end                 2017-08-15
missing_onpromotion            55706
missing_onpromotion_pct        18.57%
negative_sales_count           13
negative_sales_pct             0.00%

DATA REDUCTION PIPELINE:
  125,497,040 rows (original)
  → 22,941,656 rows (Guayas filter, -81.7%)
  → 14,745,768 rows (top-3 families, -35.7%)
  → 300,000 rows (random sample, -98.0%)
  Final: 0.24% of original dataset


In [13]:
# Data quality validation checks
print("DATA QUALITY VALIDATION")
print("=" * 70)

# Check 1: All Guayas stores present
all_stores_present = set(df_sample['store_nbr'].unique()) == set(guayas_store_nbrs)
print(f"✓ All 11 Guayas stores present: {all_stores_present}")

# Check 2: Only top-3 families
only_top3 = set(df_sample['family'].unique()) == set(top_3_families)
print(f"✓ Only top-3 families present: {only_top3}")

# Check 3: No duplicate rows
no_duplicates = df_sample.duplicated().sum() == 0
print(f"✓ No duplicate rows: {no_duplicates} (duplicates: {df_sample.duplicated().sum()})")

# Check 4: All required columns present
required_cols = ['id', 'date', 'store_nbr', 'item_nbr', 'unit_sales', 'onpromotion', 'family', 'class', 'perishable']
all_cols_present = all(col in df_sample.columns for col in required_cols)
print(f"✓ All required columns present: {all_cols_present}")

# Check 5: Date range coverage
date_range_years = pd.to_datetime(df_sample['date']).dt.year.nunique()
print(f"✓ Date range covers {date_range_years} years (2013-2017)")

# Check 6: Memory efficiency
memory_mb = df_sample.memory_usage(deep=True).sum() / 1024**2
print(f"✓ Memory usage: {memory_mb:.1f} MB (manageable for analysis)")

print("\n" + "=" * 70)
print("VALIDATION COMPLETE - Dataset ready for EDA (Day 3)")

DATA QUALITY VALIDATION
✓ All 11 Guayas stores present: True
✓ Only top-3 families present: True
✓ No duplicate rows: True (duplicates: 0)
✓ All required columns present: True
✓ Date range covers 5 years (2013-2017)
✓ Memory usage: 36.6 MB (manageable for analysis)

VALIDATION COMPLETE - Dataset ready for EDA (Day 3)


In [14]:
# Notebook completion summary
print("=" * 70)
print("NOTEBOOK COMPLETE: d02_w01_EDA_data_loading_filtering.ipynb")
print("=" * 70)

print("\nACCOMPLISHMENTS:")
print("✓ Loaded train.csv (125M rows) with Dask")
print("✓ Filtered to Guayas region (11 stores, 22.9M rows)")
print("✓ Filtered to top-3 families (14.7M rows)")
print("✓ Random sampled 300K rows (seed=42, reproducible)")
print("✓ Exported CSV and pickle to data/processed/")

print("\nFILES CREATED:")
print(f"  - guayas_sample_300k.csv (16.0 MB)")
print(f"  - guayas_sample_300k.pkl (20.6 MB)")

print("\nKEY CHARACTERISTICS:")
print(f"  Rows: 300,000")
print(f"  Stores: 11 (Guayas region)")
print(f"  Families: 3 (GROCERY I 56.9%, BEVERAGES 22.0%, CLEANING 21.1%)")
print(f"  Items: 2,296 unique")
print(f"  Date range: 2013-01-02 to 2017-08-15 (5 years)")
print(f"  Missing onpromotion: 18.57% (55,706 rows)")
print(f"  Negative sales: 13 rows (0.00%)")

print("\nDATA QUALITY NOTES:")
print("  → onpromotion NaN values: Will fill with False in Day 3")
print("  → Negative unit_sales: Will clip to 0 in Day 3")
print("  → Date column: String type, convert to datetime in Day 3")
print("  → Missing dates: Calendar gaps to fill in Day 3")

print("\nNEXT STEPS (Day 3):")
print("  1. Load guayas_sample_300k.pkl")
print("  2. Handle missing onpromotion values (fill False)")
print("  3. Clip negative unit_sales to 0")
print("  4. Fill calendar gaps (complete daily index)")
print("  5. Outlier detection and analysis")

print("\nREADY FOR DAY 3 - EDA Part 1: Quality & Preprocessing ✓")

NOTEBOOK COMPLETE: d02_w01_EDA_data_loading_filtering.ipynb

ACCOMPLISHMENTS:
✓ Loaded train.csv (125M rows) with Dask
✓ Filtered to Guayas region (11 stores, 22.9M rows)
✓ Filtered to top-3 families (14.7M rows)
✓ Random sampled 300K rows (seed=42, reproducible)
✓ Exported CSV and pickle to data/processed/

FILES CREATED:
  - guayas_sample_300k.csv (16.0 MB)
  - guayas_sample_300k.pkl (20.6 MB)

KEY CHARACTERISTICS:
  Rows: 300,000
  Stores: 11 (Guayas region)
  Families: 3 (GROCERY I 56.9%, BEVERAGES 22.0%, CLEANING 21.1%)
  Items: 2,296 unique
  Date range: 2013-01-02 to 2017-08-15 (5 years)
  Missing onpromotion: 18.57% (55,706 rows)
  Negative sales: 13 rows (0.00%)

DATA QUALITY NOTES:
  → onpromotion NaN values: Will fill with False in Day 3
  → Negative unit_sales: Will clip to 0 in Day 3
  → Date column: String type, convert to datetime in Day 3
  → Missing dates: Calendar gaps to fill in Day 3

NEXT STEPS (Day 3):
  1. Load guayas_sample_300k.pkl
  2. Handle missing onpromoti